In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
df=pd.read_csv('Input/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
netflix=df[['type','title','country','release_year','duration','listed_in','description']]
netflix=netflix[netflix.type=='Movie']
netflix=netflix[netflix.release_year>2018]
#netflix=netflix.head(999)
netflix.shape

(406, 7)

In [4]:
apiKey = os.getenv("API_IH")
print("WE HAVE APIKEY") if apiKey else print("NO APIKEY FOUND")

WE HAVE APIKEY


In [5]:
def getID(titulo):
    try:    
        url = "https://movie-database-imdb-alternative.p.rapidapi.com/"
        querystring = {"page":"1","r":"json","s":titulo}
        headers = {'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",'x-rapidapi-key': "e0036560bdmsh47b2555b69c4d3ep1b7788jsn26958ed59ecd"}
        response = requests.request("GET", url, headers=headers, params=querystring)
        response=(response.text)
        response=((response.replace(' ','').replace('"imdbID":'," ").replace(',"Type":',' ').split())[1]).replace('\"',"")
    except:pass
    return(response)

In [6]:
netflix['id']=[getID(x) for x in netflix['title']]

In [8]:
#netflix=netflix.drop[netflix['id']=='{"Response":"False","Error":"Movie not found!"}'netflix=netflix.drop[netflix['id']=='{"Response":"False","Error":"Movie not found!"}'~]
netflix = netflix[~(netflix.id == '{"Response":"False","Error":"Movie not found!"}')]
netflix = netflix[~(netflix.id == '{"Response":"False","Error":"Too many results."}')]

In [9]:
net=netflix

In [10]:
def getFromTMDB(id_tmdb,i):
    try:
        res = (requests.get(f"https://api.themoviedb.org/3/movie/{id_tmdb}?api_key={apiKey}&language=en-US")).json()[f'{i}']
    except:
        res = 'NaN'
    return res

In [12]:
for i in ['popularity','vote_count','vote_average']:
    net[f'{i}'] = [getFromTMDB(x,i) for x in net['id']]

In [13]:
net.head()

,type,title,country,release_year,duration,listed_in,description,id,popularity,vote_count,vote_average
0,Movie,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,tt9428190,5.408,4,7.6
35,Movie,Article 15,India,2019,125 min,"Dramas, International Movies, Thrillers",The grim realities of caste discrimination com...,tt10324144,9.88,75,7.6
44,Movie,American Factory: A Conversation with the Obamas,NaN,2019,10 min,Documentaries,Barack and Michelle Obama talk with directors ...,tt10937318,5.635,9,5.2
57,Movie,The World We Make,United States,2019,108 min,"Dramas, Romantic Movies",A teenage equestrian and a local football play...,tt7643622,5.005,12,6.3
58,Movie,Watchman,India,2019,93 min,"Comedies, Dramas, International Movies","Rushing to pay off a loan shark, a young man b...",tt5969358,2.97,1,2


In [19]:
net = net[~(netflix.listed_in == 'Nan')].reset_index()

In [20]:
net.head()

,index,type,title,country,release_year,duration,listed_in,description,id,popularity,vote_count,vote_average
0,0,Movie,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,tt9428190,5.408,4,7.6
1,35,Movie,Article 15,India,2019,125 min,"Dramas, International Movies, Thrillers",The grim realities of caste discrimination com...,tt10324144,9.88,75,7.6
2,44,Movie,American Factory: A Conversation with the Obamas,NaN,2019,10 min,Documentaries,Barack and Michelle Obama talk with directors ...,tt10937318,5.635,9,5.2
3,57,Movie,The World We Make,United States,2019,108 min,"Dramas, Romantic Movies",A teenage equestrian and a local football play...,tt7643622,5.005,12,6.3
4,58,Movie,Watchman,India,2019,93 min,"Comedies, Dramas, International Movies","Rushing to pay off a loan shark, a young man b...",tt5969358,2.97,1,2
